In [3]:
import sys
sys.path.insert(1, '/Users/madisonthantu/Desktop/DREAM/t-recs')
from trecs.metrics import MSEMeasurement, InteractionSpread, InteractionSpread, InteractionSimilarity, RecSimilarity, RMSEMeasurement, InteractionMeasurement
from trecs.components import Users

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from collections import defaultdict

sys.path.insert(1, '/Users/madisonthantu/Desktop/DREAM/T-RECS-RS-research')
from prelim_experiments.param_experiments.chaney_utils import (
    load_sim_results,
    graph_relative_to_ideal,
    merge_results,
    graph_metrics,
    graph_metrics_by_axis,
    graph_relative_to_global_by_axis,
    transform_relative_to_global,
    graph_histogram_metric_by_axis,
    graph_averaged_metric_by_axis
)
from wrapper.models.bubble import BubbleBurster
from src.utils import compute_constrained_clusters, create_global_user_pairs, user_topic_mapping, create_cluster_user_pairs, load_and_process_movielens, compute_embeddings
from wrapper.metrics.clustering_metrics import MeanCosineSim, MeanDistanceFromCentroid, MeanCosineSimPerCluster, MeanDistanceFromCentroidPerCluster
from prelim_experiments.param_experiments.chaney_utils import *

import warnings
warnings.simplefilter("ignore")

import itertools
import os

In [14]:
single_training_paths = []
for f in os.walk("sim_results/single_training"):
    single_training_paths.append(f[0])
single_training_paths = single_training_paths[1:]

repeated_training_paths = []
for f in os.walk("sim_results/repeated_training"):
    repeated_training_paths.append(f[0])
repeated_training_paths = repeated_training_paths[1:]

for i in range(len(single_training_paths)):
    print(single_training_paths[i])
    print(repeated_training_paths[i])
    print()

sim_results/single_training/baseline_myopic-0retraining-0probabilistic-1repeated_items_repeat_interactions
sim_results/repeated_training/xquad_binary-1retraining-0probabilistic

sim_results/single_training/xquad_binary-0retraining-0probabilistic
sim_results/repeated_training/random_recommender-1retraining-Falseprobabilistic-10random_items_per_iter

sim_results/single_training/baseline_myopic-0retraining-0probabilistic-0repeated_items_repeat_interactions
sim_results/repeated_training/baseline_myopic-1retraining-0probabilistic-1repeated_items_repeat_interactions

sim_results/single_training/random_recommender-0retraining-Falseprobabilistic-10random_items_per_iter
sim_results/repeated_training/probabilistic_recommender-1retraining-1probabilistic

sim_results/single_training/xquad_smooth-0retraining-0probabilistic
sim_results/repeated_training/xquad_smooth-1retraining-0probabilistic

sim_results/single_training/probabilistic_recommender-0retraining-1probabilistic
sim_results/repeated_train

In [ ]:
# results_path = ["sim_results/repeated_training/tester"]
# results_file = ["sim_results.pkl"]
# environment_file = ["sim_environment.pkl"]
# diagnostics_file = ["sim_diagnostics.pkl"]

# print()
# results = merge_results(results_path, results_file)
# print(results.keys())
# print(results["mse"].keys())
# print(results["mean_global_cosine_sim"]["tester_Trueretraining_Trueprobabilistic"])

# print()
# diagnostics = load_sim_results(results_path[0], diagnostics_file[0])
# print(diagnostics.keys())
# print(diagnostics["mean_global_cosine_sim"].keys())
# print(diagnostics["mean_global_cosine_sim"]["tester_Trueretraining_Trueprobabilistic"].keys())
# print(diagnostics["mean_global_cosine_sim"]["tester_Trueretraining_Trueprobabilistic"]["max"])

# print()
# environment = merge_results(results_path, environment_file)
# print(environment.keys())
# print(environment["user_cluster_centroids"].keys())
# print(environment["user_cluster_centroids"]["tester_Trueretraining_Trueprobabilistic"][0].shape)


In [9]:
results_files = ["sim_results.pkl"] * len(single_training_paths)

single_training_results = merge_results(single_training_paths, results_files)
print(single_training_results.keys())
print(single_training_results["mse"].keys())
# print(single_training_results["mean_global_cosine_sim"]["tester_Trueretraining_Trueprobabilistic"])

repeated_training_results = merge_results(repeated_training_paths, results_files)
print(repeated_training_results.keys())
print(repeated_training_results["mse"].keys())

dict_keys(['mse', 'interaction_spread', 'global_interaction_similarity', 'inter_cluster_interaction_similarity', 'intra_cluster_interaction_similarity', 'mean_global_cosine_sim', 'mean_intra_cluster_cosine_sim', 'mean_inter_cluster_cosine_sim', 'mean_cosine_sim_per_cluster', 'mean_cluster_distance_from_centroid', 'mean_global_distance_from_centroid', 'mean_distance_from_centroid_per_cluster', 'interaction_histogram', 'rmse', 'mean_novelty', 'mean_slate_topic_diversity', 'topic_interaction_histogram', 'topic_interaction_spread', 'mse_per_user'])
dict_keys(['baseline_myopic-0retraining-0probabilistic-1repeated_items_repeat_interactions-1repeated_items_repeat_interactions', 'xquad_binary_0.1alpha', 'xquad_binary_0.25alpha', 'baseline_myopic-0retraining-0probabilistic', 'random_recommender-0retraining-Falseprobabilistic-10random_items_per_iter', 'xquad_smooth_0.1alpha', 'xquad_smooth_0.25alpha', 'probabilistic_recommender-0retraining-1probabilistic-Falserepeated_items_repeat_interactions']